Stage 1: Data crawler. Massively parallel functions crawling data and storing it in Cloud Object Storage.  Obtain information from web pages or tweets and create a dataset of text data. Use FaaS backend in lithops to launch crawling process over serverless functions.

In [ ]:
import lithops
import tweepy

In [1]:
import twint


#https://github.com/twintproject/twint/wiki/User-attributes
#https://github.com/twintproject/twint/wiki/Tweet-attributes
#https://github.com/twintproject/twint/wiki/Configuration

import nest_asyncio
nest_asyncio.apply()

c = twint.Config()
c.Search = "code"
c.Limit = 10

twint.run.Search(c)

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs


KeyboardInterrupt: 

In [ ]:
# Setup Tweepy

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

NUMBER_OF_ITEMS=8
NAME_TO_ANALIZE='astrazeneca'
BUCKET_NAME='astrazeneca'

In [ ]:
public_tweets = api.retweets(1388184494235127814, 1)
print(len(public_tweets))
for tweet in public_tweets:
    print(tweet)

In [ ]:

storage = lithops.Storage()

Scrap/crawl into storage

In [ ]:
def get_tweets_replies(tweet_username, target_tweet_id):
  NUMBER_OF_ITEMS = 3

  for tweet in tweepy.Cursor(api.search,q='to:'+tweet_username, result_type='mixed', sinceId=target_tweet_id, tweet_mode='extended',).items(NUMBER_OF_ITEMS):
    print(tweet._json['full_text'])
    if hasattr(tweet, 'in_reply_to_status_id_str'):
        if (tweet.in_reply_to_status_id_str==target_tweet_id):
            print(tweet._json)
            storage.put_object(bucket='replies',
                              key=tweet._json['id_str'],
                              body=str(tweet._json))
  

In [ ]:
def get_tweets_replies_v2(conversation_id):
  cur = tweepy.Cursor(api.search, q='conversation_id='+str(conversation_id))
  print(cur)
  print(cur.items())
  for tweet in cur.items():
    print(tweet)

get_tweets_replies_v2(1388184494235127814)

In [1]:
def get_general_tweets(name, prefix, storage):
    i=0
    for tweet in tweepy.Cursor(api.search,q=prefix+name,tweet_mode='extended',since='2020-01-01').items(): # Since the start of the pandemic
        storage.put_object(BUCKET_NAME, tweet._json['id_str'], str(tweet._json))
        i+=1
    return i

In [ ]:
# tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang, tweet.source, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str, tweet.is_quote_status] for tweet in tweets]

In [ ]:
with lithops.FunctionExecutor() as fexec:
    fexec.call_async(get_general_tweets, (NAME_TO_ANALIZE, "@"))
    print(fexec.get_result())

Tweets: 
1) Pedro Sanchez
 https://twitter.com/sanchezcastejon/status/1388184494235127814 -- Jo evitaria estos que son repostes a altres tuits
 https://twitter.com/sanchezcastejon/status/1376241393396543492
 https://twitter.com/sanchezcastejon/status/1371013139987365888

2)  Boris Johnson
https://twitter.com/BorisJohnson/status/1391413912889921542
https://twitter.com/BorisJohnson/status/1363552116133023744
https://twitter.com/BorisJohnson/status/1354147835369959435

3) Joe Biden
https://twitter.com/JoeBiden/status/1386426190861197313
https://twitter.com/POTUS/status/1373037818277167112
https://twitter.com/POTUS/status/1384136220284915712
https://twitter.com/POTUS/status/1368993395172139014

4) Elon Musk
https://twitter.com/elonmusk/status/1256240766600409088


